订购量$\times$(1-亏损比)=销售量

单个商品获利 = 7月1日定价 $\times$ 7月1日订购量$\times$(1-亏损比) - 7月1日批发价 $\times$ 7月1日订购量

In [42]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

In [43]:
def get_fee(path):


    # 准备数据集
    df = pd.read_csv(path)
    df = df.drop(['单品名称', '分类名称', '单品编码', '销售类型', '是否打折销售'], axis=1)

    df['日期'] = pd.to_datetime(df['日期'])
    df = df.loc[(df['日期'] >= '2023-06-23') & (df['日期'] <= '2023-06-30')]



    # 计算平均利润
    df['平均利润'] = (df['销售单价(元/千克)'] - df['批发价格(元/千克)']) * df['销量(千克)']
    
    # 计算平均利润率，避免负利润率
    df['平均利润率'] = (df['销售单价(元/千克)'] - df['批发价格(元/千克)']) / df['批发价格(元/千克)']
    
    # 将负利润率设置为0，以避免计算负数的加成率
    df['平均利润率'] = df['平均利润率']#.apply(lambda x: max(x, 0))
    
    # 计算加成率，避免分母为0
    df['加成率'] = df['平均利润率']#.apply(lambda x: x / (1 - x) if x < 1 else float('inf'))
    
    # 计算成本加成定价
    df['成本加成定价'] = df['批发价格(元/千克)'] * (1 + df['加成率'])
    
    # 如果成本加成定价大于批发价的1.75倍，则将成本加成定价设置为批发价的1.75倍
    max_allowed_price = df['批发价格(元/千克)'] * 1.75
    df['成本加成定价'] = df['成本加成定价'].where(df['成本加成定价'] <= max_allowed_price, max_allowed_price)



    # 处理异常值
    iso_forest = IsolationForest(contamination=0.05, random_state=42)  # contamination 表示异常值的比例
    iso_forest.fit(df[['销售单价(元/千克)', '成本加成定价']])
    # 预测结果 (-1 表示异常值，1 表示正常值)
    df['anomaly'] = iso_forest.predict(df[['销售单价(元/千克)', '成本加成定价']])
    # 过滤掉异常值，只保留正常值的数据
    df_filtered = df[df['anomaly'] == 1].drop('anomaly', axis=1)



    df = df_filtered.drop(['平均利润', '平均利润率', '加成率'], axis=1)

    # 对时序数据进行聚合 resample
    df_final = df.resample('D', on='日期').agg({
        '销量(千克)': 'sum',
        '销售单价(元/千克)': 'mean',
        '批发价格(元/千克)': 'mean',
        '损耗率_单品': 'mean',    
        '成本加成定价': 'mean'
    })

    df_final['成本加成定价'].fillna(0, inplace=True)
    # 选取23年6月23日到23年6月30日的数据
    # 选取2023年6月23日到2023年6月30日的数据
    # df_final = df_final.loc[(df_final.index >= '2023-06-23') & (df_final.index <= '2023-06-30')]


    return df_final

In [44]:
df01 = get_fee("中间数据集\\多表_按单品\\白玉菇(袋).csv")
df02 = get_fee("中间数据集\\多表_按单品\\菠菜.csv")
df03 = get_fee("中间数据集\\多表_按单品\\菠菜(份).csv")
df04 = get_fee("中间数据集\\多表_按单品\\菜心.csv")
df05 = get_fee("中间数据集\\多表_按单品\\虫草花(份).csv")
df06 = get_fee("中间数据集\\多表_按单品\\高瓜(1).csv")
df07 = get_fee("中间数据集\\多表_按单品\\高瓜(2).csv")
df08 = get_fee("中间数据集\\多表_按单品\\海鲜菇(包).csv")
df09 = get_fee("中间数据集\\多表_按单品\\红椒(2).csv")
df10 = get_fee("中间数据集\\多表_按单品\\红莲藕带.csv")
df11 = get_fee("中间数据集\\多表_按单品\\红薯尖.csv")
df12 = get_fee("中间数据集\\多表_按单品\\洪湖藕带.csv")
df13 = get_fee("中间数据集\\多表_按单品\\姜蒜小米椒组合装(小份).csv")
df14 = get_fee("中间数据集\\多表_按单品\\金针菇(盒).csv")
df15 = get_fee("中间数据集\\多表_按单品\\净藕(1).csv")
df16 = get_fee("中间数据集\\多表_按单品\\菱角.csv")
df17 = get_fee("中间数据集\\多表_按单品\\螺丝椒.csv")
df18 = get_fee("中间数据集\\多表_按单品\\螺丝椒(份).csv")
df19 = get_fee("中间数据集\\多表_按单品\\木耳菜.csv")
df20 = get_fee("中间数据集\\多表_按单品\\木耳菜(份).csv")
df21 = get_fee("中间数据集\\多表_按单品\\奶白菜.csv")
df22 = get_fee("中间数据集\\多表_按单品\\七彩椒(2).csv")
df23 = get_fee("中间数据集\\多表_按单品\\青红杭椒组合装(份).csv")
df24 = get_fee("中间数据集\\多表_按单品\\青茄子(1).csv")
df25 = get_fee("中间数据集\\多表_按单品\\青线椒(份).csv")
df26 = get_fee("中间数据集\\多表_按单品\\上海青.csv")
df27 = get_fee("中间数据集\\多表_按单品\\双孢菇(盒).csv")
df28 = get_fee("中间数据集\\多表_按单品\\娃娃菜.csv")
df29 = get_fee("中间数据集\\多表_按单品\\外地茼蒿.csv")
df30 = get_fee("中间数据集\\多表_按单品\\芜湖青椒(1).csv")
df31 = get_fee("中间数据集\\多表_按单品\\西兰花.csv")
df32 = get_fee("中间数据集\\多表_按单品\\西峡花菇(1).csv")
df33 = get_fee("中间数据集\\多表_按单品\\鲜木耳(份).csv")
df34 = get_fee("中间数据集\\多表_按单品\\苋菜.csv")
df35 = get_fee("中间数据集\\多表_按单品\\小米椒(份).csv")
df36 = get_fee("中间数据集\\多表_按单品\\小青菜(1).csv")
df37 = get_fee("中间数据集\\多表_按单品\\小皱皮(份).csv")
df38 = get_fee("中间数据集\\多表_按单品\\蟹味菇与白玉菇双拼(盒).csv")
df39 = get_fee("中间数据集\\多表_按单品\\野生粉藕.csv")
df40 = get_fee("中间数据集\\多表_按单品\\圆茄子(2).csv")
df41 = get_fee("中间数据集\\多表_按单品\\云南生菜.csv")
df42 = get_fee("中间数据集\\多表_按单品\\云南生菜(份).csv")
df43 = get_fee("中间数据集\\多表_按单品\\云南油麦菜.csv")
df44 = get_fee("中间数据集\\多表_按单品\\云南油麦菜(份).csv")
df45 = get_fee("中间数据集\\多表_按单品\\长线茄.csv")
df46 = get_fee("中间数据集\\多表_按单品\\枝江青梗散花.csv")
df47 = get_fee("中间数据集\\多表_按单品\\竹叶菜.csv")
df48 = get_fee("中间数据集\\多表_按单品\\紫茄子(1).csv")
df49 = get_fee("中间数据集\\多表_按单品\\紫茄子(2).csv")


C:\Users\jack\AppData\Local\Temp\ipykernel_14452\4026270506.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['成本加成定价'].fillna(0, inplace=True)
C:\Users\jack\AppData\Local\Temp\ipykernel_14452\4026270506.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [45]:
df49

,销量(千克),销售单价(元/千克),批发价格(元/千克),损耗率_单品,成本加成定价
日期,,,,,
2023-06-23,18.258,6.0,3.81,6.07,6.0
2023-06-24,13.573,6.0,3.80,6.07,6.0
2023-06-25,16.448,6.0,3.80,6.07,6.0
2023-06-26,16.395,6.0,3.80,6.07,6.0
2023-06-27,4.095,6.0,3.80,6.07,6.0
2023-06-28,6.075,6.0,3.80,6.07,6.0
2023-06-29,5.369,6.0,3.80,6.07,6.0
2023-06-30,14.365,6.0,3.43,6.07,6.0


In [46]:
def find_yueshu(df):

    xiao1 = min(df["销量(千克)"])
    xiao2 = max(df["销量(千克)"])
    jia1 = min(df["成本加成定价"])
    jia2 = max(df["成本加成定价"])
    
    result = [xiao1, xiao2, jia1, jia2]
    return result

In [47]:
result_1 = find_yueshu(df01)
result_2 = find_yueshu(df02)
result_3 = find_yueshu(df03)
result_4 = find_yueshu(df04)
result_5 = find_yueshu(df05)
result_6 = find_yueshu(df06)
result_7 = find_yueshu(df07)
result_8 = find_yueshu(df08)
result_9 = find_yueshu(df09)
result_10 = find_yueshu(df10)
result_11 = find_yueshu(df11)
result_12 = find_yueshu(df12)
result_13 = find_yueshu(df13)
result_14 = find_yueshu(df14)
result_15 = find_yueshu(df15)
result_16 = find_yueshu(df16)
result_17 = find_yueshu(df17)
result_18 = find_yueshu(df18)
result_19 = find_yueshu(df19)
result_20 = find_yueshu(df20)
result_21 = find_yueshu(df21)
result_22 = find_yueshu(df22)
result_23 = find_yueshu(df23)
result_24 = find_yueshu(df24)
result_25 = find_yueshu(df25)
result_26 = find_yueshu(df26)
result_27 = find_yueshu(df27)
result_28 = find_yueshu(df28)
result_29 = find_yueshu(df29)
result_30 = find_yueshu(df30)
result_31 = find_yueshu(df31)
result_32 = find_yueshu(df32)
result_33 = find_yueshu(df33)
result_34 = find_yueshu(df34)
result_35 = find_yueshu(df35)
result_36 = find_yueshu(df36)
result_37 = find_yueshu(df37)
result_38 = find_yueshu(df38)
result_39 = find_yueshu(df39)
result_40 = find_yueshu(df40)
result_41 = find_yueshu(df41)
result_42 = find_yueshu(df42)
result_43 = find_yueshu(df43)
result_44 = find_yueshu(df44)
result_45 = find_yueshu(df45)
result_46 = find_yueshu(df46)
result_47 = find_yueshu(df47)
result_48 = find_yueshu(df48)
result_49 = find_yueshu(df49)

In [48]:
result = pd.DataFrame({
    "白玉菇(袋)": result_1, "菠菜": result_2, "菠菜(份)": result_3,
    "菜心": result_4, "虫草花(份)": result_5, "高瓜(1)": result_6,
    "高瓜(2)": result_7, "海鲜菇(包)": result_8, "红椒(2)": result_9,
    "红莲藕带": result_10, "红薯尖": result_11, "洪湖藕带": result_12,
    "姜蒜小米椒组合装(小份)": result_13, "金针菇(盒)": result_14, "净藕(1)": result_15,
    "菱角": result_16, "螺丝椒": result_17, "螺丝椒(份)": result_18,
    "木耳菜": result_19, "木耳菜(份)": result_20, "奶白菜": result_21,
    "七彩椒(2)": result_22, "青红杭椒组合装(份)": result_23, "青茄子(1)": result_24,
    "青线椒(份)": result_25, "上海青": result_26, "双孢菇(盒)": result_27,
    "娃娃菜": result_28, "外地茼蒿": result_29, "芜湖青椒(1)": result_30,
    "西兰花": result_31, "西峡花菇(1)": result_32, "鲜木耳(份)": result_33,
    "苋菜": result_34, "小米椒(份)": result_35, "小青菜(1)": result_36,
    "小皱皮(份)": result_37, "蟹味菇与白玉菇双拼(盒)": result_38, "野生粉藕": result_39,
    "圆茄子(2)": result_40, "云南生菜": result_41, "云南生菜(份)": result_42,
    "云南油麦菜": result_43, "云南油麦菜(份)": result_44, "长线茄": result_45,
    "枝江青梗散花": result_46, "竹叶菜": result_47, "紫茄子(1)": result_48,
    "紫茄子(2)": result_49
    }, index=['销量最小值', '销量最大值', '定价最小值', '定价最大值']).T

result['单品名称'] = result.index
result.index = range(0, len(result))
result

,销量最小值,销量最大值,定价最小值,定价最大值,单品名称
0,1.000,1.000,4.100000,6.247500,白玉菇(袋)
1,1.275,3.252,14.000000,14.000000,菠菜
2,0.000,20.000,0.000000,5.990000,菠菜(份)
3,0.000,4.112,0.000000,6.000000,菜心
4,1.000,4.000,2.300000,3.800000,虫草花(份)
5,1.973,5.927,10.909091,16.000000,高瓜(1)
6,0.000,2.089,0.000000,18.000000,高瓜(2)
7,4.000,13.000,2.400000,3.000000,海鲜菇(包)
8,1.325,3.950,17.333333,20.000000,红椒(2)
9,0.218,1.037,9.200000,9.200000,红莲藕带


In [49]:
result['销量约束min'] = result['销量最小值'] * 0.8
result['销量约束max'] = result['销量最大值'] * 1.2
result['定价约束min'] = result['定价最小值'] * 0.8
result['定价约束max'] = result['定价最大值'] * 1.2
result = result[['单品名称', '销量约束min', '销量约束max', '定价约束min', '定价约束max']]
result

,单品名称,销量约束min,销量约束max,定价约束min,定价约束max
0,白玉菇(袋),0.8000,1.2000,3.280000,7.497000
1,菠菜,1.0200,3.9024,11.200000,16.800000
2,菠菜(份),0.0000,24.0000,0.000000,7.188000
3,菜心,0.0000,4.9344,0.000000,7.200000
4,虫草花(份),0.8000,4.8000,1.840000,4.560000
5,高瓜(1),1.5784,7.1124,8.727273,19.200000
6,高瓜(2),0.0000,2.5068,0.000000,21.600000
7,海鲜菇(包),3.2000,15.6000,1.920000,3.600000
8,红椒(2),1.0600,4.7400,13.866667,24.000000
9,红莲藕带,0.1744,1.2444,7.360000,11.040000


In [50]:
kuisun = pd.read_excel("0_赛题\\附件4.xlsx", sheet_name="Sheet1").drop("单品编码", axis=1)
kuisun

,单品名称,损耗率(%)
0,牛首生菜,4.39
1,四川红香椿,10.46
2,西峡花菇(1),10.80
3,本地小毛白菜,0.18
4,白菜苔,8.78
...,...,...
246,海鲜菇(袋)(3),1.30
247,鲜粽叶(袋)(2),0.00
248,鲜粽叶(袋)(3),9.43
249,虫草花(盒)(2),11.13


In [51]:
kuisun['最小补货量'] = 2.5 / (1 - kuisun['损耗率(%)'] / 100)
kuisun.columns = ['单品名称', '损耗率', '最小补货量']

In [52]:
kuisun.shape

(251, 3)

In [53]:
result_final = pd.merge(result, kuisun, on='单品名称', how='left')
result_final

,单品名称,销量约束min,销量约束max,定价约束min,定价约束max,损耗率,最小补货量
0,白玉菇(袋),0.8000,1.2000,3.280000,7.497000,6.57,2.675800
1,菠菜,1.0200,3.9024,11.200000,16.800000,18.51,3.067861
2,菠菜(份),0.0000,24.0000,0.000000,7.188000,9.43,2.760296
3,菜心,0.0000,4.9344,0.000000,7.200000,13.70,2.896871
4,虫草花(份),0.8000,4.8000,1.840000,4.560000,9.43,2.760296
5,高瓜(1),1.5784,7.1124,8.727273,19.200000,29.25,3.533569
6,高瓜(2),0.0000,2.5068,0.000000,21.600000,9.43,2.760296
7,海鲜菇(包),3.2000,15.6000,1.920000,3.600000,0.00,2.500000
8,红椒(2),1.0600,4.7400,13.866667,24.000000,9.43,2.760296
9,红莲藕带,0.1744,1.2444,7.360000,11.040000,16.63,2.998681


In [54]:
result_final['成本'] = \
    pd.DataFrame([
    df01['批发价格(元/千克)'][-1], df02['批发价格(元/千克)'][-1], df03['批发价格(元/千克)'][-1], df04['批发价格(元/千克)'][-1], 
    df05['批发价格(元/千克)'][-1], df06['批发价格(元/千克)'][-1], df07['批发价格(元/千克)'][-1], df08['批发价格(元/千克)'][-1], 
    df09['批发价格(元/千克)'][-1], df10['批发价格(元/千克)'][-1], df11['批发价格(元/千克)'][-1], df12['批发价格(元/千克)'][-1], 
    df13['批发价格(元/千克)'][-1], df14['批发价格(元/千克)'][-1], df15['批发价格(元/千克)'][-1], df16['批发价格(元/千克)'][-1], 
    df17['批发价格(元/千克)'][-1], df18['批发价格(元/千克)'][-1], df19['批发价格(元/千克)'][-1], df20['批发价格(元/千克)'][-1],
    df21['批发价格(元/千克)'][-1], df22['批发价格(元/千克)'][-1], df23['批发价格(元/千克)'][-1], df24['批发价格(元/千克)'][-1],
    df25['批发价格(元/千克)'][-1], df26['批发价格(元/千克)'][-1], df27['批发价格(元/千克)'][-1], df28['批发价格(元/千克)'][-1],
    df29['批发价格(元/千克)'][-1], df30['批发价格(元/千克)'][-1], df31['批发价格(元/千克)'][-1], df32['批发价格(元/千克)'][-1],
    df33['批发价格(元/千克)'][-1], df34['批发价格(元/千克)'][-1], df35['批发价格(元/千克)'][-1], df36['批发价格(元/千克)'][-1],
    df37['批发价格(元/千克)'][-1], df38['批发价格(元/千克)'][-1], df39['批发价格(元/千克)'][-1], df40['批发价格(元/千克)'][-1],
    df41['批发价格(元/千克)'][-1], df42['批发价格(元/千克)'][-1], df43['批发价格(元/千克)'][-1], df44['批发价格(元/千克)'][-1],
    df45['批发价格(元/千克)'][-1], df46['批发价格(元/千克)'][-1], df47['批发价格(元/千克)'][-1], df48['批发价格(元/千克)'][-1],
    df49['批发价格(元/千克)'][-1]
    ])

result_final

C:\Users\jack\AppData\Local\Temp\ipykernel_14452\3245875812.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df01['批发价格(元/千克)'][-1], df02['批发价格(元/千克)'][-1], df03['批发价格(元/千克)'][-1], df04['批发价格(元/千克)'][-1],
C:\Users\jack\AppData\Local\Temp\ipykernel_14452\3245875812.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df05['批发价格(元/千克)'][-1], df06['批发价格(元/千克)'][-1], df07['批发价格(元/千克)'][-1], df08['批发价格(元/千克)'][-1],
C:\Users\jack\AppData\Local\Temp\ipykernel_14452\3245875812.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels

,单品名称,销量约束min,销量约束max,定价约束min,定价约束max,损耗率,最小补货量,成本
0,白玉菇(袋),0.8000,1.2000,3.280000,7.497000,6.57,2.675800,3.29
1,菠菜,1.0200,3.9024,11.200000,16.800000,18.51,3.067861,9.66
2,菠菜(份),0.0000,24.0000,0.000000,7.188000,9.43,2.760296,4.07
3,菜心,0.0000,4.9344,0.000000,7.200000,13.70,2.896871,4.62
4,虫草花(份),0.8000,4.8000,1.840000,4.560000,9.43,2.760296,2.60
5,高瓜(1),1.5784,7.1124,8.727273,19.200000,29.25,3.533569,11.67
6,高瓜(2),0.0000,2.5068,0.000000,21.600000,9.43,2.760296,13.69
7,海鲜菇(包),3.2000,15.6000,1.920000,3.600000,0.00,2.500000,1.95
8,红椒(2),1.0600,4.7400,13.866667,24.000000,9.43,2.760296,12.72
9,红莲藕带,0.1744,1.2444,7.360000,11.040000,16.63,2.998681,5.29


In [55]:
result.to_excel("中间数据集\\3_2_7月1日单品寻优的约束条件.xlsx")